# Iris Tensorflow Keras

This notebook walks you through iris species classification model training in PEDL on the popular [Iris flower dataset](https://en.wikipedia.org/wiki/Iris_flower_data_set) using the TensorFlow Keras machine learning library.

In [ ]:
# Test importing PEDL.  In PEDL is properly installed, you will see no output.
import pedl

In [ ]:
# Replace with the IP address of the PEDL master.
pedl_master = '<master IP>'

# Run an Experiment

First, we will explore the components of a PEDL experiment; namely, the dataset specification, the model definition, and experiment configurations.

## Model Directory
- `data.py`: The data loading interface implementation
- `model_def.py`: The model definition
- `__init__.py`: The entrypoint for PEDL to resolve the experiment trial interface
- `*.yaml` configuration files that each govern an individual experiment run

Let's look at the contents of the model directory:

In [ ]:
!ls iris_tf_keras

<a id='data_py'></a>
### data.py
Take a look at the data loading functionality in `data.py`.  The `make_data_loaders` function is a required function that specifies the training and validation datasets to use in an experiment, while `download_data` is an optional performance efficiency API which, if implemented, causes PEDL to download and cache data once per machine rather than per process.

In [ ]:
!cat iris_tf_keras/data.py

### model_def.py
Drill in and view the model definition file.  Look for the implementation of PEDL's `TFKerasTrial` interface.  This is the interface between PEDL and TensorFlow Keras, which ultimately enables the ML Engineer to leverage PEDL's distributed hyperparameter search in a shared runtime without having to worry about these distributed system concerns.

In [ ]:
!cat -n iris_tf_keras/model_def.py

### \_\_init\_\_.py
Given that a PEDL model definition is a Python package, `__init__.py` is the entrypoint that exposes the trial interface implementation and data loading methods.

In [ ]:
!cat iris_tf_keras/__init__.py

### const.yaml
For our first PEDL experiment, we'll run a model training job with fixed hyperparameters.  Note the following sections:
- `description`: A short description of the experiment
- `data`: A section for user to provide custom key value pairs.  Here we specify where the training and validation datasets reside.  Note its usage in [data.py](#data.py).
- `hyperparameters`: Hyperparameter values that are injected into the trial at runtime. There are constant values for this configuration
- `searcher`: hyperparameter search algorithm for the experiment

In [ ]:
!cat iris_tf_keras/const.yaml

## Submit Experiment

In [ ]:
!pedl -m {pedl_master} experiment create iris_tf_keras/const.yaml iris_tf_keras/

Once the experiment completes (which may take a few minutes if PEDL agents have to start up), look at the experiment page to see the single completed trial.  Note the 90+% accuracy.

# Adaptive Hyperparameter Search
### adaptive.yaml

Next, let's run an experiment with the same model definition, but we'll leverage PEDL's adaptive hyperparameter search to efficiently determine the hyperparameter values that yield the highest categorical accuracy.  Note that the hyperparameters in the experiment configuration are specified as ranges as opposed to fixed values as in our [first experiment](#const.yaml).

In [ ]:
!cat iris_tf_keras/adaptive.yaml

## Submit Experiment

In [ ]:
!pedl -m {pedl_master} experiment create iris_tf_keras/adaptive.yaml iris_tf_keras/

During and after the experiment run, you can view the best (highest) categorical accuracy that PEDL's adaptive search finds over time:

![adaptive](../img/iris_tf_keras_adaptive.png)

When the experiment finishes, observe that we hit 100% accuracy on the 30 test set observations, an improvement over the fixed hyperparameter experiment above.  From the PEDL experiment detail page, you can drill in to a particular trial and view the hyperparameter values used.  You can also access the saved checkpoint of your best-performing model and load it for real-time or batch inference as described in the Keras documentation [here](https://www.tensorflow.org/versions/r1.15/api_docs/python/tf/keras/models/load_model).